In [0]:
import scipy.io as io
import numpy as np

In [0]:
train_datamat = io.loadmat('indian_pines_Train_patch_3.mat')
train_data = train_datamat['train_patch']
train_labels = train_datamat['train_labels'].transpose()

train_labels1 = train_labels.ravel()
trainLabels = np.zeros((train_labels1.size, train_labels1.max()+1))
trainLabels[np.arange(train_labels1.size),train_labels1] = 1

In [0]:
val_datamat = io.loadmat('indian_pines_Val_patch_3.mat')
val_data = val_datamat['val_patch']
val_labels = val_datamat['val_labels'].transpose()

val_labels1 = val_labels.ravel()
valLabels = np.zeros((val_labels1.size, val_labels1.max()+1))
valLabels[np.arange(val_labels1.size),val_labels1] = 1

In [0]:
test_datamat = io.loadmat('indian_pines_Test_patch_3.mat')
test_data = test_datamat['test_patch']
test_labels = test_datamat['test_labels'].transpose()

test_labels1 = test_labels.ravel()
testLabels = np.zeros((test_labels1.size, test_labels1.max()+1))
testLabels[np.arange(test_labels1.size),test_labels1] = 1

In [0]:
train_data.shape

(1600, 220, 3, 3)

In [0]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Lambda, Flatten, Input, Conv2D, Conv1D, Reshape, Concatenate, Dropout

cn1 = Conv1D(filters = 20, kernel_size = 3, strides = 1, activation="relu")
cn2 = Conv1D(filters = 20, kernel_size = 3, strides = 1, activation="relu")
cn3 = Conv1D(filters = 10, kernel_size = 3, strides = 1, activation="relu")
cn4 = Conv1D(filters = 5, kernel_size = 5, strides = 1, activation="relu")

inp = Input(shape=[10, 22, 3*3])
out = []
for i in range(10):
    reshape1 = Lambda(lambda x: x[:,i,:,:])(inp)  
    conv1 = cn1(reshape1)
    conv2 = cn2(conv1)
    conv3 = cn3(conv2)
    conv4 = cn4(conv3)
    reshape2 = Reshape((60, 1), input_shape = conv4.shape)(conv4)
    out.append(reshape2)
outConc = keras.layers.concatenate(out)
parallelModel = Model(inputs=[inp], outputs=outConc)

net = Sequential()
net.add(Conv2D(220, kernel_size = 1, activation="relu", data_format="channels_first"))
net.add(Reshape((10, 22, 3*3)))
net.add(parallelModel)
net.add(Flatten())
net.add(Dense(100, activation='relu'))
net.add(Dropout(0.5))
net.add(Dense(9, activation='softmax'))

In [0]:
# from keras.utils import plot_model
# plot_model(net, to_file='net.png')

In [0]:
from keras.optimizers import adam

ad = adam(lr=0.0005)
net.compile(optimizer=ad, loss='categorical_crossentropy', metrics=['accuracy'])
net.fit(train_data, trainLabels, validation_data=(val_data, valLabels), batch_size = 200, epochs=1000)

Train on 1600 samples, validate on 200 samples
Epoch 1/1000
1600/1600 [==============================] - 3s 2ms/step - loss: 0.1012 - acc: 0.9650 - val_loss: 0.2454 - val_acc: 0.9350
Epoch 2/1000
1600/1600 [==============================] - 1s 510us/step - loss: 0.0906 - acc: 0.9675 - val_loss: 0.2796 - val_acc: 0.9100
Epoch 3/1000
1600/1600 [==============================] - 1s 543us/step - loss: 0.0897 - acc: 0.9681 - val_loss: 0.2182 - val_acc: 0.9350
Epoch 4/1000
1600/1600 [==============================] - 1s 549us/step - loss: 0.0781 - acc: 0.9688 - val_loss: 0.2256 - val_acc: 0.9400
Epoch 5/1000
1600/1600 [==============================] - 1s 506us/step - loss: 0.0759 - acc: 0.9731 - val_loss: 0.2215 - val_acc: 0.9450
Epoch 6/1000
1600/1600 [==============================] - 1s 557us/step - loss: 0.0972 - acc: 0.9669 - val_loss: 0.2129 - val_acc: 0.9250
Epoch 7/1000
1600/1600 [==============================] - 1s 561us/step - loss: 0.1115 - acc: 0.9531 - val_loss: 0.2871 - val_a

In [0]:
net.evaluate(test_data, testLabels)

7434/7434 [==============================] - 1s 153us/step


[0.5310749423972347, 0.9296475652407856]

In [0]:
net.evaluate(val_data, valLabels)

200/200 [==============================] - 0s 167us/step


[0.26612766325473786, 0.955]

In [0]:
net.evaluate(train_data, trainLabels)

1600/1600 [==============================] - 0s 148us/step


[4.272651058101928e-05, 1.0]